In [ ]:
import re
import sys
import glob
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
from time import ctime
from sklearn.model_selection import KFold
from joblib import Parallel, delayed
import multiprocessing
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn import preprocessing
register_matplotlib_converters()
sns.set_style('darkgrid')

In [ ]:
train_frags = glob.glob("./train_fft/*")
test_frags = glob.glob("./test_fft/*")
train = pd.read_csv("train.csv")
Y = pd.Series(0, index=np.arange(len(train_frags)))

i = 0
for file in train_frags:
    start = './train_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]
    t2e = int(train.loc[train['segment_id'] == int(seg_id)]['time_to_eruption'].values)
    Y.iloc[i] = t2e
    i = i + 1

test_id = pd.Series(0, index=np.arange(len(test_frags)))    
i = 0
for file in test_frags:
    start = './test_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]

    test_id.iloc[i] = seg_id
    i = i + 1

In [ ]:
from sklearn import preprocessing
timescaler = preprocessing.StandardScaler().fit(Y.to_numpy().reshape(-1,1))
Y = pd.Series(timescaler.transform(Y.to_numpy().reshape(-1,1)).reshape(-1))

In [ ]:
all_pca = pd.read_csv("pca_50.csv")

In [ ]:
X = all_pca[:len(train_frags)]
X_test = all_pca[-len(test_frags):]

In [ ]:
from sklearn.linear_model._bayes import ARDRegression
from sklearn.ensemble._weight_boosting import AdaBoostRegressor
from sklearn.linear_model._bayes import BayesianRidge
from sklearn.tree._classes import DecisionTreeRegressor 
from sklearn.linear_model._coordinate_descent import ElasticNetCV
from sklearn.ensemble._forest import ExtraTreesRegressor
from sklearn.gaussian_process._gpr import GaussianProcessRegressor
from sklearn.linear_model._glm.glm import GeneralizedLinearRegressor
from sklearn.ensemble._gb import GradientBoostingRegressor
from sklearn.ensemble._hist_gradient_boosting.gradient_boosting import HistGradientBoostingRegressor
from sklearn.linear_model._huber import HuberRegressor
from sklearn.isotonic import IsotonicRegression 
from sklearn.neighbors._regression import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model._least_angle import LarsCV
from sklearn.linear_model._coordinate_descent import LassoCV
from sklearn.linear_model._least_angle import LassoLarsCV
from sklearn.linear_model._least_angle import LassoLarsIC
from sklearn.linear_model._base import LinearRegression
from sklearn.svm._classes import LinearSVR
from sklearn.neural_network._multilayer_perceptron import MLPRegressor
from sklearn.svm._classes import NuSVR
from sklearn.linear_model._omp import OrthogonalMatchingPursuitCV
from sklearn.linear_model._passive_aggressive import PassiveAggressiveRegressor
from sklearn.neighbors._regression import RadiusNeighborsRegressor
from sklearn.ensemble._forest import RandomForestRegressor
from sklearn.linear_model._ridge import RidgeCV
from sklearn.linear_model._stochastic_gradient import SGDRegressor
from sklearn.svm._classes import SVR
from sklearn.linear_model._glm.glm import TweedieRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

estimators = [('ard', ARDRegression()),
              ('ada', AdaBoostRegressor()),
              ('brr', BayesianRidge()),
              ('dtr', DecisionTreeRegressor()),
              ('enc', ElasticNetCV()),
              ('etr', ExtraTreesRegressor()),
              ('gpr', GaussianProcessRegressor()),
              ('glr', GeneralizedLinearRegressor()),
              ('gbr', GradientBoostingRegressor()),
              ('hgb', HistGradientBoostingRegressor()),
              ('hur', HuberRegressor()),
              ('knr', KNeighborsRegressor()),
              ('ker', KernelRidge()),
              ('lar', LarsCV()),
              ('las', LassoCV()),
              ('llc', LassoLarsCV()),
              ('lli', LassoLarsIC()),
              ('lir', LinearRegression()),
              ('lsv', LinearSVR(max_iter=100000)),
              ('mlp', MLPRegressor(max_iter=10000)),
              ('nsv', NuSVR(max_iter=100000)),
              ('par', PassiveAggressiveRegressor(max_iter=10000)),
              ('omp', OrthogonalMatchingPursuitCV()),
              ('rnr', RadiusNeighborsRegressor()),
              ('rfr', RandomForestRegressor()),
              ('rdg', RidgeCV()),
              ('sgd', SGDRegressor(max_iter=10000)),
              ('svr', SVR(max_iter=100000)),
              ('twr', TweedieRegressor(max_iter=10000))]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
Y_test.to_csv('Y_pca_50.csv', index=False)
for est in estimators:
    name,model = est
    smodel = model
    smodel.fit(X_train,Y_train)
    pred = smodel.predict(X_test)
    
    preddf = pd.DataFrame({
        "pred": pred
    })
    sname = name + '_pca_50.csv'
    preddf.to_csv(sname, index=False)